# **Project 2:**

In [46]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from src.Ward import Ward, initialize_wards
from src.Patient import Patient, initialize_patients
from src.simulation import simulation_loop, run_simulations,compute_gradient
from src.__pycache__.Skibedy import *

ImportError: cannot import name 'compute_gradient' from 'src.simulation' (c:\Users\MGAJ\OneDrive - Danmarks Tekniske Universitet\DTU\Kandidat\4_Semester\Stokastisk Simulering\StochasticSim\Projects\Project 2\src\simulation.py)

## initialization

In [45]:
F_bed_capacity = 30 ##### Number to optimize #####

#patient type from 'A' to 'F'
patient_types = ['A', 'B', 'C', 'D', 'E', 'F']
bed_capacity = np.array([55,40,30,20,20, F_bed_capacity])
arrivals_pr_day = np.array([14.5,11.0,8.0,6.5,5.0, 13.0])
mean_length_of_stay = np.array([2.9,4.0,4.5,1.4,3.9, 2.0])
urgency_points = np.array([7,5,2,10,5, 0])

#reloaction probability
relocation_probability = np.array([
    [0.0, 0.05, 0.10, 0.05, 0.80, 0.00],
    [0.2, 0, 0.50, 0.15, 0.15, 0.00],
    [0.30, 0.20, 0, 0.20, 0.30, 0.00],
    [0.35, 0.30, 0.05, 0, 0.3, 0.00],
    [0.20, 0.10, 0.60 ,0.10, 0, 0.00],
    [0.20, 0.20, 0.20, 0.20, 0.20 ,0]
    ])

#Dataframe containing all the data (indexed by patient type)
df = pd.DataFrame({
    'Patient Type': patient_types,
    'Bed Capacity': bed_capacity,
    'Arrivals per day': arrivals_pr_day,
    'Mean Length of Stay': mean_length_of_stay,
    'Urgency Points': urgency_points
    }).set_index('Patient Type')

In [22]:
#sampling functions. Takes type as input and returns a sample with the corresponding distribution
arrival_interval_function = lambda type: np.random.exponential(1/df["Arrivals per day"][type])
occupancy_time_function = lambda type: np.random.exponential(df["Mean Length of Stay"][type])

# Optimize F beds

Build a simulation model that simulates the patient flow in the hospital as a function of the bed
distribution and the aforementioned parameters

In [23]:
total_time = 31 #simulate 1 month
wards = initialize_wards(df)
performance = run_simulations(total_time, wards, relocation_probability, arrival_interval_function, occupancy_time_function, n_simulations = 10, verbose = False)

{A Ward with 55 beds and 7 urgency points.: {'Occupied probability': 0.046538194410187056,
  'Estimated admissions': 470.49999999999994,
  'Estimated rejections': 23.9,
  'Estimated relocations': 9.7},
 B Ward with 40 beds and 5 urgency points.: {'Occupied probability': 0.22214864544041604,
  'Estimated admissions': 296.9,
  'Estimated rejections': 84.8,
  'Estimated relocations': 53.39999999999999},
 C Ward with 30 beds and 2 urgency points.: {'Occupied probability': 0.3400051182261519,
  'Estimated admissions': 210.29999999999998,
  'Estimated rejections': 109.4,
  'Estimated relocations': 61.099999999999994},
 D Ward with 20 beds and 10 urgency points.: {'Occupied probability': 0.03978308778358209,
  'Estimated admissions': 225.5,
  'Estimated rejections': 9.6,
  'Estimated relocations': 6.199999999999999},
 E Ward with 20 beds and 5 urgency points.: {'Occupied probability': 0.29407544005904374,
  'Estimated admissions': 147.4,
  'Estimated rejections': 62.69999999999999,
  'Estimat

In [24]:
#convert performance to dataframe
df_performance = pd.DataFrame(performance).T
#exclude last row
df_performance = df_performance.iloc[:-1]
#change index
df_performance.index = [ward.type for ward in wards]
df_performance["Estimated lost"] = df_performance["Estimated rejections"] - df_performance["Estimated relocations"]

In [25]:
#create a stacked bar chart of the occupancy of the wards
import plotly.express as px
fig = px.bar(df_performance[["Estimated admissions", "Estimated relocations", "Estimated lost"]])
fig.update_layout(barmode='stack',
                    title='Occupancy of the wards after 1 month',
                    xaxis_title='Ward',
                    yaxis_title='Number of patients')
fig.show()

 Create a new ward (F
∗
) in the system and allocate a minimal number of the current bed
resources to the new ward. Ensure that at least 95% of the type F
∗ patients are hospitalized in
Ward F
∗
. Use the ”urgency points” from Table 1 to balance the solution (prioritize wards that
need beds more than other wards).

In [26]:
F_bed_capacity = 0
F_occupied_prob = 1.0
penalties_F = []
F_occupied_probabilities = []
while F_occupied_prob > 0.01:
    #set F bed capacity in the dataframe
    df.loc['F', 'Bed Capacity'] = F_bed_capacity
    wards = initialize_wards(df) #list of wards
    performance = run_simulations(total_time, wards, relocation_probability, arrival_interval_function, occupancy_time_function, n_simulations = 10, verbose = False)
    F_ward = wards[-1]
    F_occupied_prob = performance[F_ward]["Occupied probability"]
    print("Probability of rejection",F_occupied_prob)
    penalties_F.append(performance["Weighted penalty"])
    F_occupied_probabilities.append(F_occupied_prob)
    F_bed_capacity += 1

Probability of rejection 0.9999999999999999
Probability of rejection 0.9612065275822314
Probability of rejection 0.9259480393715274
Probability of rejection 0.8781165823634738
Probability of rejection 0.8505456380830763
Probability of rejection 0.8051990601747185
Probability of rejection 0.754775557224597
Probability of rejection 0.7356781287088274
Probability of rejection 0.6994909526465539
Probability of rejection 0.6576447161475598
Probability of rejection 0.6122362639501262
Probability of rejection 0.591834203120289
Probability of rejection 0.5462035834046458
Probability of rejection 0.4988608889538873
Probability of rejection 0.4876022876335899
Probability of rejection 0.43040888356171675
Probability of rejection 0.40655015729626887
Probability of rejection 0.3726381124738624
Probability of rejection 0.31990475577562066
Probability of rejection 0.3276163304167671
Probability of rejection 0.28491318514386677
Probability of rejection 0.2303797180105767
Probability of rejection 0.217

In [27]:
#Number of beds required to ensure that 95% of the arrivals in F are admitted
F_bed_capacity_optimal = np.argmax(np.array(F_occupied_probabilities) < 0.05)
print("Number of beds required to ensure that 95% of the arrivals in F are admitted", F_bed_capacity_optimal)

Number of beds required to ensure that 95% of the arrivals in F are admitted 31


In [28]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = np.arange(0, F_bed_capacity), y = penalties_F, name='Weighted penalty'))
fig.update_layout(title = "Penalty and rejection rate as a function of F bed capacity",
                  xaxis_title = "F bed capacity",
                  yaxis_title = "Weighted penalty")
fig.add_trace(go.Scatter(x = np.arange(0, F_bed_capacity), y = F_occupied_probabilities, yaxis = 'y2', name='Rejection probability'))
fig.update_layout(yaxis2 = dict(title = "Rejection probability", overlaying = 'y', side = 'right',
                                ))# Find the index where F_occupied_probabilities is closest to 0.05
index = (np.abs(np.array(F_occupied_probabilities) - 0.05)).argmin()

# Add a point at that index
fig.add_trace(go.Scatter(x=[np.arange(0, F_bed_capacity)[index]], 
                         y=[F_occupied_probabilities[index]], 
                         mode='markers', 
                         marker=dict(size=10, color='red'), 
                         name='5% Rejection'))
fig.show()

At some point increasing the number of beds in a ward will not decrease the penalty.

# Sensitivity analysis
Test the system’s sensitivity to the length-of-stay distribution by replacing the exponential
distribution with the log-normal distribution. Test the new distribution by gradually increasing
the variance (e.g. σ
2
i = 2/µ2
i
, 3/µ2
i
and 4/µ2
i
)

In [29]:
#sampling functions. Takes type as input and returns a sample with the corresponding distribution
#Test the system’s sensitivity to the length-of-stay distribution by replacing the exponential
#distribution with the log-normal distribution. Test the new distribution by gradually increasing
#the variance 

arrival_interval_function = lambda type: np.random.exponential(1/df["Arrivals per day"][type])
occupancy_time_function = lambda var_scale : lambda type: np.random.lognormal(mean = np.log(df["Mean Length of Stay"][type]), sigma = np.sqrt(var_scale/df["Mean Length of Stay"][type]**2))

In [30]:
var_scales = np.linspace(1, 10, 10)
penalties = []
df.loc['F', 'Bed Capacity'] = F_bed_capacity_optimal
for var_scale in var_scales:
    wards = initialize_wards(df) #list of wards
    occupancy_time_function_var = occupancy_time_function(var_scale)
    performance = run_simulations(total_time, wards, relocation_probability, arrival_interval_function, occupancy_time_function_var, n_simulations = 10, verbose = False)
    penalties.append(performance["Weighted penalty"])

In [31]:
#plot the results with PyPlot
fig = go.Figure()
fig.add_trace(go.Scatter(x = var_scales, y = penalties, name='Weighted penalty'))
fig.update_layout(title = "Weighted penalty as variance increases",
                  xaxis_title = "μ^2 * σ^2",
                  yaxis_title = "Weighted penalty")
fig.show()

Test the system’s sensitivity to the distribution of beds in the hospital.

In [33]:
#gradient of the weighted penalty as a function of the number of beds in each ward
wards = initialize_wards(df) #list of wards
arrival_interval_function = lambda type: np.random.exponential(1/df["Arrivals per day"][type])
occupancy_time_function = lambda type: np.random.exponential(df["Mean Length of Stay"][type])

In [39]:
# Get the gradients for each ward
gradients = compute_gradient(total_time, wards, relocation_probability, arrival_interval_function, occupancy_time_function, n_simulations = 10, verbose = False)

# Create a color array, setting the color to 'red' if the gradient is negative, 'blue' otherwise
colors = ['red' if gradient < 0 else 'blue' for gradient in gradients]

fig = go.Figure()
fig.add_trace(go.Bar(x=[ward.type for ward in wards], y=gradients, marker_color=colors))
fig.update_layout(title="Gradient of the weighted penalty as a function of the number of beds in each ward",
                  xaxis_title="Ward",
                  yaxis_title="Partial derivative of the weighted penalty")
fig.show()